In [2]:
import numpy as np
import pandas as pd
import json
from geopy.geocoders import Nominatim
import folium
import requests # library to handle requests for postal codes in wikipedia
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
print("libraries imported")

libraries imported


# Import data and prepare a dataframe of Dallas neighborhoods

Next, we need to import the Postal Code (zip code) for the Dallas city along with its neighborhoods and their lat/longs

In [3]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_21def94ec5cc4908b0c31f1dc24c75ba = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='abcd',
    ibm_auth_endpoint="https://efs",
    config=Config(signature_version='obcda'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_abcd.get_object(Bucket='capstonefinal-donotdelete-pr-gs4myuipwvirzd',Key='texas_zips.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

raw_zip = pd.read_csv(body)
raw_zip.head()


,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,75001,32.96000,-96.83847,Addison,TX,Texas,True,NaN,12414,1250.2,48113,Dallas,{'48113':100},False,False,America/Chicago
1,75002,33.08966,-96.60751,Allen,TX,Texas,True,NaN,63140,655.6,48085,Collin,{'48085':100},False,False,America/Chicago
2,75006,32.96188,-96.89701,Carrollton,TX,Texas,True,NaN,46364,1065.0,48113,Dallas,{'48113':100},False,False,America/Chicago
3,75007,33.00462,-96.89714,Carrollton,TX,Texas,True,NaN,51624,1709.9,48121,Denton,"{'48113':5.79,'48121':94.21}",False,False,America/Chicago
4,75009,33.34028,-96.75033,Celina,TX,Texas,True,NaN,8785,35.5,48085,Collin,"{'48085':94.8,'48121':5.2}",False,False,America/Chicago


Next, we need to import the Postal Code (zip code) for the Dallas city along with its neighborhoods and their lat/longs

In [4]:
# Drop zcta, county_fips, all_county_weights, imprecise, military, timezone columns
raw_zip.drop(['zcta','parent_zcta','density','county_fips','all_county_weights','imprecise','military','timezone'], axis=1, inplace=True)
raw_zip=raw_zip[['state_name', 'state_id', 'county_name', 'city', 'zip', 'lat', 'lng', 'population']]

# Rename city-> neighborhood, county-> city, lat/long, state_name->state
raw_zip.rename(columns={'lat':'latitude','lng':'longitude','city':'neighborhood'}, inplace=True)
raw_zip.rename(columns={'county_name':'city','state_name':'state'}, inplace=True)
#raw_zip.set_index("city")

In [5]:
# create a new dataframe only with Dallas neighborhoods
dallas_zip=raw_zip[raw_zip.city == 'Dallas']
dallas_zip.set_index("city")

print(dallas_zip.dtypes)
print(dallas_zip.shape)
#print(raw_zip.columns)
dallas_zip.head()       # this is the dataframe to work with

state            object
state_id         object
city             object
neighborhood     object
zip               int64
latitude        float64
longitude       float64
population        int64
dtype: object
(84, 8)


,state,state_id,city,neighborhood,zip,latitude,longitude,population
0,Texas,TX,Dallas,Addison,75001,32.96000,-96.83847,12414
2,Texas,TX,Dallas,Carrollton,75006,32.96188,-96.89701,46364
7,Texas,TX,Dallas,Coppell,75019,32.96329,-96.98553,38666
18,Texas,TX,Dallas,Irving,75038,32.87458,-96.99758,27802
19,Texas,TX,Dallas,Irving,75039,32.88752,-96.94225,11032


Now to render a map of Neighborhood (cities) in the County of Dallas

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent dallas_explorer, as shown below.

In [6]:
address = 'Dallas, TX'
geolocator = Nominatim(user_agent="dallas_explorer")
location = geolocator.geocode(address)
d_latitude = location.latitude
d_longitude = location.longitude
print('The geographical coordinate of Dallas city is {}, {}.'.format(d_latitude, d_longitude))

The geographical coordinate of Dallas city is 32.7762719, -96.7968559.


In [7]:
# create map of Dallas using latitude and longitude values
map_dallas = folium.Map(location=[d_latitude, d_longitude], zoom_start=10)

# add markers to map
for lat, lng, city, neighborhood in zip(dallas_zip['latitude'], dallas_zip['longitude'], dallas_zip['city'], dallas_zip['neighborhood']):
    label = '{}, {}'.format(neighborhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dallas)  
    
map_dallas

# Explore Neighborhoods (cities) venues in the county of Dallas using Foursquare

Explore and cluster the cities in Dallas -
Define Foursquare Credentials and Version and some parameters

In [8]:
CLIENT_ID = '4V0IGGK1WLK33B0WCVIYJBASHJPAC44RTALPOWU14XJHDDOG'
CLIENT_SECRET = 'YQNBTI5VSPNN0BNP2QNHYLHUSRNT1CDHB5TV2PLJO3P33AVG' # your Foursquare Secret
VERSION = '20180605'                # Foursquare API version

radius=3000                         # default radius is 500m from ll
LIMIT = 500                         # default LIMIT 500 venues per neighborhood


Create a function to repeat the same process for exploring venues to all the cities of Dallas county

In [9]:
def getNearbyVenues(names, latitudes, longitudes):   # names -> cities in Dallas county, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?categoryId= 4d4b7105d754a06374d81259&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,                # radius = 3000 as set in previous cell
            LIMIT)                 # LIMIT  =  500 as set in previous cell
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function for each city and store it in a new dataframe called dallas_venues

In [10]:
dallas_venues = getNearbyVenues(names=dallas_zip['neighborhood'],
                                   latitudes=dallas_zip['latitude'],
                                   longitudes=dallas_zip['longitude']
                                  )

Addison
Carrollton
Coppell
Irving
Irving
Garland
Garland
Garland
Garland
Garland
Sachse
Grand Prairie
Grand Prairie
Grand Prairie
Irving
Irving
Irving
Irving
Richardson
Richardson
Rowlett
Rowlett
Cedar Hill
Desoto
Duncanville
Lancaster
Duncanville
Hutchins
Lancaster
Mesquite
Mesquite
Seagoville
Wilmer
Balch Springs
Mesquite
Sunnyvale
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas


Let's do a sanity check on the resulting data

In [ ]:
##print(dallas_venues.shape)
##dallas_venues.head()


Let's check how many venues were returned for each city (neighborhood)

In [11]:
dallas_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Addison,100,100,100,100,100,100
Balch Springs,37,37,37,37,37,37
Carrollton,100,100,100,100,100,100
Cedar Hill,48,48,48,48,48,48
Coppell,75,75,75,75,75,75
Dallas,3485,3485,3485,3485,3485,3485
Desoto,43,43,43,43,43,43
Duncanville,114,114,114,114,114,114
Garland,315,315,315,315,315,315


In [12]:
# Let's find out how many unique categories can be curated from all the returned venues - debug 
print('There are {} uniques categories.'.format(len(dallas_venues['Venue Category'].unique())))

There are 88 uniques categories.



Analyze Each Neighborhood

In [13]:
# one hot encoding
dallas_onehot = pd.get_dummies(dallas_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dallas_onehot['Neighborhood'] = dallas_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dallas_onehot.columns[-1]] + list(dallas_onehot.columns[:-1])
dallas_onehot = dallas_onehot[fixed_columns]

dallas_onehot.shape   # examine the new dataframe size

(5465, 89)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
dallas_grouped = dallas_onehot.groupby('Neighborhood').mean().reset_index()
dallas_grouped                         # 
dallas_grouped.shape                   # Let's confirm the new size

(19, 89)

Let's print each neighborhood along with the top 10 most common venues

In [15]:
num_top_venues = 20     # normally only the top 10 

for hood in dallas_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dallas_grouped[dallas_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Addison----
                   venue  freq
0     Mexican Restaurant  0.11
1     Italian Restaurant  0.09
2    American Restaurant  0.08
3           Burger Joint  0.05
4                 Bakery  0.04
5             Restaurant  0.04
6                  Diner  0.04
7             Steakhouse  0.04
8            Wings Joint  0.03
9       Sushi Restaurant  0.03
10           Pizza Place  0.03
11        Sandwich Place  0.03
12    Seafood Restaurant  0.03
13  Fast Food Restaurant  0.03
14             BBQ Joint  0.03
15        Breakfast Spot  0.02
16   Fried Chicken Joint  0.02
17   Japanese Restaurant  0.02
18         Deli / Bodega  0.02
19       Thai Restaurant  0.02


----Balch Springs----
                         venue   freq
0         Fast Food Restaurant  0.243
1          Fried Chicken Joint  0.135
2                  Pizza Place  0.108
3           Chinese Restaurant  0.108
4                         Food  0.081
5                   Taco Place  0.054
6           Mexican Restaurant  0.054
7    

First, let's write a function to sort the venues in descending order.

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 20     # normally only the top 10 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dallas_grouped['Neighborhood']

for ind in np.arange(dallas_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dallas_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted   # now to eyeball our data

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Addison,Mexican Restaurant,Italian Restaurant,American Restaurant,Burger Joint,Bakery,Restaurant,Steakhouse,Diner,Seafood Restaurant,...,BBQ Joint,Fast Food Restaurant,Sandwich Place,Pizza Place,Wings Joint,Japanese Restaurant,Deli / Bodega,New American Restaurant,Breakfast Spot,Thai Restaurant
1,Balch Springs,Fast Food Restaurant,Fried Chicken Joint,Pizza Place,Chinese Restaurant,Food,Taco Place,Mexican Restaurant,Burger Joint,Sandwich Place,...,Café,Diner,Donut Shop,Bakery,Asian Restaurant,Food Court,Food Stand,Food Truck,Fondue Restaurant,French Restaurant
2,Carrollton,Fast Food Restaurant,Mexican Restaurant,Korean Restaurant,Pizza Place,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Burger Joint,Café,...,Bakery,Vietnamese Restaurant,Fried Chicken Joint,Cajun / Creole Restaurant,Thai Restaurant,Asian Restaurant,Donut Shop,Diner,Creperie,Italian Restaurant
3,Cedar Hill,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Fried Chicken Joint,Pizza Place,Burger Joint,Seafood Restaurant,Donut Shop,Breakfast Spot,...,Food,Sandwich Place,Café,Italian Restaurant,Wings Joint,Southern / Soul Food Restaurant,Sushi Restaurant,Bakery,Tex-Mex Restaurant,BBQ Joint
4,Coppell,Pizza Place,Fast Food Restaurant,American Restaurant,Sandwich Place,Mexican Restaurant,Donut Shop,Tex-Mex Restaurant,Burger Joint,Café,...,Indian Restaurant,Bakery,Food,Cajun / Creole Restaurant,Japanese Restaurant,Mediterranean Restaurant,Vietnamese Restaurant,Greek Restaurant,Diner,Deli / Bodega
5,Dallas,Mexican Restaurant,Fast Food Restaurant,Pizza Place,American Restaurant,Sandwich Place,Burger Joint,Fried Chicken Joint,Taco Place,Italian Restaurant,...,Bakery,BBQ Joint,Chinese Restaurant,New American Restaurant,Steakhouse,Restaurant,Donut Shop,Breakfast Spot,Café,Thai Restaurant
6,Desoto,Pizza Place,Donut Shop,Fast Food Restaurant,American Restaurant,Fried Chicken Joint,Sandwich Place,Mexican Restaurant,Chinese Restaurant,Seafood Restaurant,...,Fish & Chips Shop,Restaurant,Snack Place,Wings Joint,Bakery,Dim Sum Restaurant,Fondue Restaurant,Food,Food Court,Filipino Restaurant
7,Duncanville,Fast Food Restaurant,Pizza Place,American Restaurant,Mexican Restaurant,Fried Chicken Joint,Seafood Restaurant,Chinese Restaurant,Italian Restaurant,Sandwich Place,...,Wings Joint,BBQ Joint,Burger Joint,Breakfast Spot,Sushi Restaurant,Bagel Shop,Bakery,Taco Place,Cajun / Creole Restaurant,Restaurant
8,Garland,Fast Food Restaurant,Mexican Restaurant,Pizza Place,American Restaurant,Chinese Restaurant,Burger Joint,Sandwich Place,Donut Shop,Fried Chicken Joint,...,BBQ Joint,Food,Restaurant,Seafood Restaurant,Wings Joint,Breakfast Spot,Bakery,Vietnamese Restaurant,Sushi Restaurant,Italian Restaurant
9,Grand Prairie,Fast Food Restaurant,Mexican Restaurant,Pizza Place,Bakery,Fried Chicken Joint,Sandwich Place,American Restaurant,Taco Place,Donut Shop,...,Wings Joint,Burger Joint,Chinese Restaurant,Breakfast Spot,Italian Restaurant,Seafood Restaurant,Restaurant,Gastropub,Food Stand,Food Truck


In [18]:
neighborhoods_venues_sorted.to_csv(r'Dallas Venues.csv')

# Now to explore Asian food in the using Foursquare

In [19]:
def getNearbyAsianVenues(names, latitudes, longitudes):   # names -> cities in Dallas county, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?categoryId=4bf58dd8d48988d142941735&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,                # radius = 500 as set in previous cell
            LIMIT)                 # LIMIT = 100 as set in previous cell
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
dallas_asian_venues = getNearbyAsianVenues(names=dallas_zip['neighborhood'],
                                   latitudes=dallas_zip['latitude'],
                                   longitudes=dallas_zip['longitude']
                                  )

Addison
Carrollton
Coppell
Irving
Irving
Garland
Garland
Garland
Garland
Garland
Sachse
Grand Prairie
Grand Prairie
Grand Prairie
Irving
Irving
Irving
Irving
Richardson
Richardson
Rowlett
Rowlett
Cedar Hill
Desoto
Duncanville
Lancaster
Duncanville
Hutchins
Lancaster
Mesquite
Mesquite
Seagoville
Wilmer
Balch Springs
Mesquite
Sunnyvale
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas
Dallas


In [21]:
dallas_asian_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Addison,39,39,39,39,39,39
Balch Springs,5,5,5,5,5,5
Carrollton,68,68,68,68,68,68
Cedar Hill,3,3,3,3,3,3
Coppell,9,9,9,9,9,9
Dallas,1250,1250,1250,1250,1250,1250
Desoto,3,3,3,3,3,3
Duncanville,18,18,18,18,18,18
Garland,66,66,66,66,66,66


In [22]:
# one hot encoding
dallas_asian_onehot = pd.get_dummies(dallas_asian_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dallas_asian_onehot['Neighborhood'] = dallas_asian_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dallas_asian_onehot.columns[-1]] + list(dallas_asian_onehot.columns[:-1])
dallas_asian_onehot = dallas_asian_onehot[fixed_columns]

dallas_asian_onehot.shape   # examine the new dataframe size

(1759, 38)

In [23]:
dallas_asian_grouped = dallas_asian_onehot.groupby('Neighborhood').mean().reset_index()
dallas_asian_grouped                         # 
dallas_asian_grouped.shape                   # Let's confirm the new size

(18, 38)

In [24]:
num_top_venues = 20     # normally only the top 10 

for hood in dallas_asian_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dallas_asian_grouped[dallas_asian_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 3})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Addison----
                            venue   freq
0                Asian Restaurant  0.436
1              Chinese Restaurant  0.154
2                 Thai Restaurant  0.128
3             Japanese Restaurant  0.103
4                Sushi Restaurant  0.077
5           Vietnamese Restaurant  0.051
6                    Noodle House  0.026
7               Korean Restaurant  0.026
8                     Salad Place  0.000
9                      Poke Place  0.000
10               Ramen Restaurant  0.000
11                     Restaurant  0.000
12            American Restaurant  0.000
13                 Sandwich Place  0.000
14             Seafood Restaurant  0.000
15         Peking Duck Restaurant  0.000
16                     Soup Place  0.000
17            Szechuan Restaurant  0.000
18           Taiwanese Restaurant  0.000
19  Vegetarian / Vegan Restaurant  0.000


----Balch Springs----
                            venue  freq
0              Chinese Restaurant   0.6
1                As

In [25]:
def return_most_common_asian_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
num_top_venues = 20     # normally only the top 10 

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_asian_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_asian_venues_sorted['Neighborhood'] = dallas_asian_grouped['Neighborhood']

for ind in np.arange(dallas_asian_grouped.shape[0]):
    neighborhoods_asian_venues_sorted.iloc[ind, 1:] = return_most_common_asian_venues(dallas_asian_grouped.iloc[ind, :], num_top_venues)

neighborhoods_asian_venues_sorted   # now to eyeball our data

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Addison,Asian Restaurant,Chinese Restaurant,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Korean Restaurant,Noodle House,Wings Joint,...,Himalayan Restaurant,Fried Chicken Joint,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Café,Burmese Restaurant,Buffet,Bakery
1,Balch Springs,Chinese Restaurant,Asian Restaurant,Wings Joint,Fast Food Restaurant,Japanese Restaurant,Indian Restaurant,Himalayan Restaurant,Fried Chicken Joint,Food Truck,...,Diner,Korean Restaurant,Dim Sum Restaurant,Café,Burmese Restaurant,Buffet,Bakery,Karaoke Bar,Mongolian Restaurant,Vietnamese Restaurant
2,Carrollton,Korean Restaurant,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Vietnamese Restaurant,Thai Restaurant,Ramen Restaurant,Japanese Restaurant,Karaoke Bar,...,Noodle House,Wings Joint,Filipino Restaurant,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Himalayan Restaurant,Café,Burmese Restaurant
3,Cedar Hill,Asian Restaurant,Sushi Restaurant,Chinese Restaurant,Wings Joint,Fast Food Restaurant,Japanese Restaurant,Indian Restaurant,Himalayan Restaurant,Fried Chicken Joint,...,Filipino Restaurant,Dim Sum Restaurant,Diner,Korean Restaurant,Café,Burmese Restaurant,Buffet,Bakery,Karaoke Bar,Mongolian Restaurant
4,Coppell,Vietnamese Restaurant,Chinese Restaurant,Asian Restaurant,Thai Restaurant,Sushi Restaurant,Peking Duck Restaurant,Japanese Restaurant,Wings Joint,Filipino Restaurant,...,Fried Chicken Joint,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Café,Burmese Restaurant,Buffet,Bakery,Indian Restaurant
5,Dallas,Asian Restaurant,Chinese Restaurant,Sushi Restaurant,Thai Restaurant,Japanese Restaurant,Vietnamese Restaurant,Ramen Restaurant,Korean Restaurant,Noodle House,...,Fast Food Restaurant,Seafood Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Taiwanese Restaurant,New American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Buffet
6,Desoto,Chinese Restaurant,Thai Restaurant,Wings Joint,Fast Food Restaurant,Japanese Restaurant,Indian Restaurant,Himalayan Restaurant,Fried Chicken Joint,Food Truck,...,Dim Sum Restaurant,Diner,Korean Restaurant,Café,Burmese Restaurant,Buffet,Bakery,Asian Restaurant,Karaoke Bar,Mongolian Restaurant
7,Duncanville,Chinese Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Mongolian Restaurant,Burmese Restaurant,Buffet,Café,Bakery,...,Diner,Fast Food Restaurant,Filipino Restaurant,Food Truck,Fried Chicken Joint,Himalayan Restaurant,Indian Restaurant,Japanese Restaurant,Dim Sum Restaurant,Wings Joint
8,Garland,Chinese Restaurant,Vietnamese Restaurant,Asian Restaurant,Sushi Restaurant,Thai Restaurant,Korean Restaurant,Fast Food Restaurant,Indian Restaurant,Himalayan Restaurant,...,Food Truck,Filipino Restaurant,Wings Joint,Diner,Japanese Restaurant,Café,Burmese Restaurant,Buffet,Bakery,Dim Sum Restaurant
9,Grand Prairie,Chinese Restaurant,Asian Restaurant,Japanese Restaurant,Filipino Restaurant,Sushi Restaurant,Café,Wings Joint,Indian Restaurant,Himalayan Restaurant,...,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Korean Restaurant,Burmese Restaurant,Buffet,Bakery,Karaoke Bar,Mongolian Restaurant


In [27]:
neighborhoods_asian_venues_sorted.to_csv(r'Dallas Asian Venues.csv')

# Visualization of Cluster venues

In [28]:
kclusters = 5           # set number of clusters

# new data frame for grouping by neighborhood
dallas_asian_grouped_clustering = dallas_asian_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dallas_asian_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:18]

array([4, 1, 3, 4, 0, 0, 1, 0, 0, 1, 4, 0, 1, 1, 0, 1, 3, 2], dtype=int32)

In [29]:
# add clustering labels
#backup_neighborhoods_asian_venues_sorted=neighborhoods_asian_venues_sorted # debug - create a backup 
neighborhoods_asian_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dallas_merged = dallas_zip

# merge dallas_asian_grouped with dallas_zip to add latitude/longitude for each neighborhood
dallas_merged = dallas_merged.join(neighborhoods_asian_venues_sorted.set_index('Neighborhood'), on='neighborhood')
dallas_merged.dropna(axis = 0, inplace = True)             # incased NaN shows up after merging
dallas_merged['Cluster Labels'] = dallas_merged['Cluster Labels'].astype('int64') # Cluster Labels must be int

print(dallas_merged.dtypes)
dallas_merged                      # check the columns!

state                      object
state_id                   object
city                       object
neighborhood               object
zip                         int64
latitude                  float64
longitude                 float64
population                  int64
Cluster Labels              int64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
11th Most Common Venue     object
12th Most Common Venue     object
13th Most Common Venue     object
14th Most Common Venue     object
15th Most Common Venue     object
16th Most Common Venue     object
17th Most Common Venue     object
18th Most Common Venue     object
19th Most Common Venue     object
20th Most Common Venue     object
dtype: object


,state,state_id,city,neighborhood,zip,latitude,longitude,population,Cluster Labels,1st Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Texas,TX,Dallas,Addison,75001,32.96000,-96.83847,12414,4,Asian Restaurant,...,Himalayan Restaurant,Fried Chicken Joint,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Café,Burmese Restaurant,Buffet,Bakery
2,Texas,TX,Dallas,Carrollton,75006,32.96188,-96.89701,46364,3,Korean Restaurant,...,Noodle House,Wings Joint,Filipino Restaurant,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Himalayan Restaurant,Café,Burmese Restaurant
7,Texas,TX,Dallas,Coppell,75019,32.96329,-96.98553,38666,0,Vietnamese Restaurant,...,Fried Chicken Joint,Food Truck,Dim Sum Restaurant,Fast Food Restaurant,Diner,Café,Burmese Restaurant,Buffet,Bakery,Indian Restaurant
18,Texas,TX,Dallas,Irving,75038,32.87458,-96.99758,27802,0,Asian Restaurant,...,Indian Restaurant,Bakery,Wings Joint,Dim Sum Restaurant,Diner,Fast Food Restaurant,Café,Food Truck,Fried Chicken Joint,Burmese Restaurant
19,Texas,TX,Dallas,Irving,75039,32.88752,-96.94225,11032,0,Asian Restaurant,...,Indian Restaurant,Bakery,Wings Joint,Dim Sum Restaurant,Diner,Fast Food Restaurant,Café,Food Truck,Fried Chicken Joint,Burmese Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Texas,TX,Dallas,Dallas,75251,32.91901,-96.77193,2331,0,Asian Restaurant,...,Fast Food Restaurant,Seafood Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Taiwanese Restaurant,New American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Buffet
156,Texas,TX,Dallas,Dallas,75253,32.67499,-96.61287,18450,0,Asian Restaurant,...,Fast Food Restaurant,Seafood Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Taiwanese Restaurant,New American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Buffet
157,Texas,TX,Dallas,Dallas,75254,32.94562,-96.80137,23253,0,Asian Restaurant,...,Fast Food Restaurant,Seafood Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Taiwanese Restaurant,New American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Buffet
158,Texas,TX,Dallas,Dallas,75270,32.78119,-96.80224,0,0,Asian Restaurant,...,Fast Food Restaurant,Seafood Restaurant,Diner,Sandwich Place,Fried Chicken Joint,Taiwanese Restaurant,New American Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Buffet


In [30]:
# create map of Dallas using latitude and longitude values
#map_clusters = folium.Map(location=[d_latitude, d_longitude], zoom_start=10)
map_dallas = folium.Map(location=[d_latitude, d_longitude], zoom_start=10)   # try adding cluster to the map_dallas. Was map_clusters

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dallas_merged['latitude'], dallas_merged['longitude'], dallas_merged['neighborhood'], dallas_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_dallas)         #fill_opacity=0.7).add_to(map_clusters)
       
map_dallas
#map_clusters

End of Battle of Neighborhoods - code section